# Hyperparameter Tuning

## Goal

The goal of this experiment is to find the best combination of hyperparameters across a range of different regressor models and select the best model.

## Experiment

We conduct the experiment with following decisions:

- A time window of 1 year is used per study area, covering each year of recording.
- We use the dataset created in the previous experiment.
- We perform hyperparameter tuning for the following classifiers:
  - Extreme Learning Machine
  - K-Nearest Neighbors
  - Linear Regression fitted with Stochastic Gradient Descent
  - Support Vector Machine
  - Extra Trees
  - Histogram-based Gradient Boosting
  - Random Forest
  - eXtreme Gradient Boosting
- Bayesian Optimization with Optuna is used to find the best hyperparameters, using F1 score with stratified 5-fold cross-validation per iteration as the objective.
- The number of trials is set to 100 for models.

We load the dataset and perform the hyperparameter tuning for each model:

In [1]:
# Define the metric and load the data
import pandas as pd
from sklearn.metrics import f1_score, make_scorer

# Define the metric
f1_scorer = make_scorer(f1_score)

# Load the data
data = pd.read_csv("../data/processed/compositing/data.csv")
target = pd.read_csv("../data/processed/compositing/target.csv")["0"]

In [2]:
# Create save folder and wrapper functions
from pathlib import Path
from typing import Any

save_folder = "../models/"
Path(save_folder).mkdir(parents=True, exist_ok=True)


def _suggest_categorical(*args: Any, **kwargs: Any) -> tuple:  # noqa: ANN401
    return "suggest_categorical", args, kwargs


def _suggest_discrete_uniform(*args: Any, **kwargs: Any) -> tuple:  # noqa: ANN401
    return "suggest_discrete_uniform", args, kwargs


def _suggest_float(*args: Any, **kwargs: Any) -> tuple:  # noqa: ANN401
    return "suggest_float", args, kwargs


def _suggest_int(*args: Any, **kwargs: Any) -> tuple:  # noqa: ANN401
    return "suggest_int", args, kwargs

In [3]:
# Extreme Learning Machine
from logging import INFO
from sys import stdout

from loguru import logger
from skelm import ELMClassifier
from slc.models import hyperparam_search

logger.configure(handlers=[{"sink": stdout, "format": "{message}", "level": INFO}])

elm_model = ELMClassifier(random_state=42)
search_space = [
    _suggest_float("alpha", 1e-8, 1e5, log=True),
    _suggest_categorical("include_original_features", [True, False]),
    _suggest_categorical("ufunc", ["tanh", "sigm", "relu", "lin"]),
    _suggest_categorical("n_neurons", [1, None]),
    _suggest_float("density", 0.01, 0.99),
]

elm_pipe, elm_study = hyperparam_search(
    elm_model,
    search_space,
    data,
    target,
    f1_scorer,
    n_trials=100,
    save_folder=save_folder,
    random_state=42,
)

elm_pipe

Files already exist, skipping search: ../models/ELMClassifier_study.pkl, ../models/ELMClassifier.pkl


Pipeline(steps=[('imputer', KNNImputer()), ('scaler', StandardScaler()),
                ('pca', PCA(n_components=6)),
                ('model',
                 ELMClassifier(alpha=4.121386090770351,
                               density=0.3367158032124471,
                               include_original_features=True, random_state=42,
                               ufunc='relu'))])

In [4]:
# K-Nearest Neighbour
from sklearn.neighbors import KNeighborsClassifier
from slc.models import hyperparam_search

knn_model = KNeighborsClassifier(n_jobs=-1)
search_space = [
    _suggest_int("n_neighbors", 1, 100),
    _suggest_categorical("weights", ["uniform", "distance"]),
    _suggest_categorical("algorithm", ["auto", "ball_tree", "kd_tree", "brute"]),
]

knn_pipe, knn_study = hyperparam_search(
    knn_model,
    search_space,
    data,
    target,
    f1_scorer,
    n_trials=100,
    save_folder=save_folder,
    random_state=42,
)

knn_pipe

Files already exist, skipping search: ../models/KNeighborsClassifier_study.pkl, ../models/KNeighborsClassifier.pkl


Pipeline(steps=[('imputer', KNNImputer()), ('scaler', StandardScaler()),
                ('model',
                 KNeighborsClassifier(algorithm='brute', n_jobs=-1,
                                      n_neighbors=50, weights='distance'))])

In [5]:
# Linear Regression
from sklearn.linear_model import SGDClassifier

sgd_model = SGDClassifier(random_state=42)
search_space = [
    _suggest_categorical(
        "loss",
        [
            "squared_epsilon_insensitive",
            "modified_huber",
            "log_loss",
            "perceptron",
            "squared_error",
            "squared_hinge",
            "epsilon_insensitive",
            "huber",
            "hinge",
        ],
    ),
    _suggest_float("alpha", 1e-6, 1e5, log=True),
    _suggest_float("l1_ratio", 0, 1),
    _suggest_int("max_iter", 1, 10000),
]

sgd_pipe, sgd_study = hyperparam_search(
    sgd_model,
    search_space,
    data,
    target,
    f1_scorer,
    n_trials=100,
    save_folder=save_folder,
    random_state=42,
)

sgd_pipe

Files already exist, skipping search: ../models/SGDClassifier_study.pkl, ../models/SGDClassifier.pkl


Pipeline(steps=[('imputer', KNNImputer()), ('scaler', StandardScaler()),
                ('model',
                 SGDClassifier(alpha=3.8892335430160876,
                               l1_ratio=0.8000360249553982, loss='log_loss',
                               max_iter=6948, random_state=42))])

In [6]:
# Support Vector Machine
from sklearn.svm import SVC

svm_model = SVC()
search_space = [
    _suggest_float("C", 1e-5, 1e5, log=True),
    _suggest_categorical("kernel", ["poly", "rbf", "sigmoid"]),
]

svm_pipe, svm_study = hyperparam_search(
    svm_model,
    search_space,
    data,
    target,
    f1_scorer,
    n_trials=100,
    save_folder=save_folder,
    random_state=42,
    always_standardize=True,
)

svm_pipe

Files already exist, skipping search: ../models/SVC_study.pkl, ../models/SVC.pkl


Pipeline(steps=[('imputer', KNNImputer()), ('scaler', StandardScaler()),
                ('pca', PCA(n_components=173)),
                ('model', SVC(C=2.6622005221586065))])

In [7]:
# Extra Trees
from sklearn.ensemble import ExtraTreesClassifier

et_model = ExtraTreesClassifier(n_jobs=-1, random_state=42)
search_space = [
    _suggest_int("n_estimators", 1, 200),
    _suggest_float("min_impurity_decrease", 1e-5, 0.5, log=True),
    _suggest_categorical("criterion", ["gini", "entropy", "log_loss"]),
]

et_pipe, et_study = hyperparam_search(
    et_model,
    search_space,
    data,
    target,
    f1_scorer,
    n_trials=100,
    save_folder=save_folder,
    random_state=42,
)

et_pipe

Files already exist, skipping search: ../models/ExtraTreesClassifier_study.pkl, ../models/ExtraTreesClassifier.pkl


Pipeline(steps=[('imputer', KNNImputer()),
                ('model',
                 ExtraTreesClassifier(criterion='entropy',
                                      min_impurity_decrease=0.0006498230136405032,
                                      n_estimators=61, n_jobs=-1,
                                      random_state=42))])

In [8]:
# Histogram-based Gradient Boosting
from sklearn.ensemble import HistGradientBoostingClassifier

hgb_model = HistGradientBoostingClassifier(random_state=42)
search_space = [
    _suggest_int("max_iter", 10, 1000),
    _suggest_float("learning_rate", 0.001, 0.5, log=True),
    _suggest_int("max_leaf_nodes", 2, 1000),
    _suggest_categorical("l2_regularization", [0, 1e-10, 1e-5, 1e-3, 1e-1, 1]),
]

hgb_pipe, hgb_study = hyperparam_search(
    hgb_model,
    search_space,
    data,
    target,
    f1_scorer,
    n_trials=100,
    save_folder=save_folder,
    random_state=42,
)

hgb_pipe

Files already exist, skipping search: ../models/HistGradientBoostingClassifier_study.pkl, ../models/HistGradientBoostingClassifier.pkl


Pipeline(steps=[('scaler', StandardScaler()),
                ('model',
                 HistGradientBoostingClassifier(l2_regularization=1,
                                                learning_rate=0.3531950291796023,
                                                max_iter=972, max_leaf_nodes=84,
                                                random_state=42))])

In [9]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(n_jobs=-1, random_state=42)
search_space = [
    _suggest_int("n_estimators", 1, 200),
    _suggest_int("max_depth", 1, 1000),
    _suggest_float("max_features", 0.1, 1.0),
    _suggest_float("min_samples_split", 1e-5, 0.5, log=True),
    _suggest_float("min_samples_leaf", 1e-5, 0.5, log=True),
    _suggest_categorical("bootstrap", [True, False]),
    _suggest_categorical(
        "criterion",
        ["gini", "entropy", "log_loss"],
    ),
]

rf_pipe, rf_study = hyperparam_search(
    rf_model,
    search_space,
    data,
    target,
    f1_scorer,
    n_trials=100,
    save_folder=save_folder,
    random_state=42,
)

rf_pipe

Resuming search from cache at trial 6.


[I 2024-09-15 23:15:47,372] Trial 6 finished with value: 0.6088881685768327 and parameters: {'do_standardize': False, 'do_pca': True, 'n_components': 252, 'n_estimators': 153, 'max_depth': 562, 'max_features': 0.7938704619591049, 'min_samples_split': 0.0020908876411396384, 'min_samples_leaf': 0.002859603931944107, 'bootstrap': True, 'criterion': 'log_loss'}. Best is trial 5 with value: 0.6222554818740337.
[I 2024-09-15 23:15:57,046] Trial 7 finished with value: 0.610180064655425 and parameters: {'do_standardize': False, 'do_pca': True, 'n_components': 145, 'n_estimators': 152, 'max_depth': 229, 'max_features': 0.1692819188459137, 'min_samples_split': 0.00022989704335273252, 'min_samples_leaf': 5.7223444071741426e-05, 'bootstrap': True, 'criterion': 'entropy'}. Best is trial 5 with value: 0.6222554818740337.
[I 2024-09-15 23:16:36,106] Trial 8 finished with value: 0.6403649267396547 and parameters: {'do_standardize': False, 'do_pca': False, 'n_estimators': 180, 'max_depth': 319, 'max_fe

In [ ]:
# eXtreme Gradient Boosting
from slc.models import hyperparam_search
from xgboost import XGBClassifier

xgb_model = XGBClassifier(n_jobs=-1, random_state=42)
search_space = [
    _suggest_int("n_estimators", 10, 200),
    _suggest_int("max_depth", 1, 20),
    _suggest_float("learning_rate", 0.001, 0.5, log=True),
    _suggest_float("gamma", 0, 0.5),
    _suggest_int("min_child_weight", 1, 11),
    _suggest_int("reg_alpha", 40, 180),
    _suggest_float("reg_lambda", 0, 1),
    _suggest_float("colsample_bytree", 0.5, 1),
]

xgb_pipe, xgb_study = hyperparam_search(
    xgb_model,
    search_space,
    data,
    target,
    f1_scorer,
    n_trials=100,
    save_folder=save_folder,
    random_state=42,
)

xgb_pipe

In [ ]:
# Plot the hyperparam search scores during search
import matplotlib.pyplot as plt
import scienceplots  # noqa: F401

from slc.visualize import save_fig

plt.style.use("science")

studies = [
    elm_study,
    knn_study,
    sgd_study,
    svm_study,
    et_study,
    hgb_study,
    rf_study,
    xgb_study,
]

fig, axs = plt.subplots(2, 4, figsize=(20, 10))

for ax, study in zip(axs.flat, studies, strict=False):
    trials = study.trials_dataframe()
    ax.plot(trials["value"], label="Score")
    ax.set_title(study.study_name)
    ax.set_xlabel("Trial")
    ax.set_ylabel("F1 Score")
    ax.axhline(study.best_value, color="g", linestyle="--", label="Best score")
    ax.set_ylim(0, 1)
    ax.legend()

save_fig(fig, "../reports/figures/hyperparameter_tuning/Hyperparameter Tuning.svg")

In above figure you can see the hyperparameter tuning process for each model. The best F1 Score of each model is between 0.6 and 0.7, while the F1 Scores during the tuning processes vary greatly. For example the K-Nearest Neighbors classifier mostly produces F1 Scores above 0.5 and the Linear Regression (SGDClassifier) frequently approaches and even reaches F1 Scores of 0. We compute the F1 Score, Accuracy and Kappa for each model:

In [ ]:
# Evaluate the tuned models
from collections import defaultdict

import pandas as pd
from sklearn.metrics import accuracy_score, cohen_kappa_score, f1_score
from sklearn.model_selection import StratifiedKFold, cross_validate
from tqdm.notebook import tqdm

csv_path = "../reports/hyperparameter_tuning.csv"
scoring = {
    "F1 Score": make_scorer(f1_score),
    "Accuracy": make_scorer(accuracy_score),
    "Kappa": make_scorer(cohen_kappa_score),
}

tuned_models = [
    elm_pipe,
    knn_pipe,
    sgd_pipe,
    svm_pipe,
    rf_pipe,
    et_pipe,
    hgb_pipe,
    xgb_pipe,
]

if not Path(csv_path).exists():
    # Create columns
    model_names = [model.steps[-1][1].__class__.__name__ for model in tuned_models]
    columns = defaultdict(list)
    columns["Model"] = model_names

    # Cross validate default and tuned models
    for model in tqdm(tuned_models):
        cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
        cv_result = cross_validate(
            model, data, target, cv=cv, scoring=scoring, n_jobs=-1
        )

        for metric in scoring:
            columns[metric].append(cv_result[f"test_{metric}"].mean())

    # Create dataframe
    results = pd.DataFrame(columns)
    results = results.set_index("Model")
    results.to_csv(csv_path)
else:
    results = pd.read_csv(csv_path, index_col="Model")

results